<a href="https://colab.research.google.com/github/SondipPoulSingh/Deep-Learning/blob/main/BIRDS_450_SPECIES_IMAGE_CLASSIFICATIONipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#https://www.kaggle.com/datasets/gpiosenka/100-bird-species
! mkdir -p ~/.kaggle
! cp kaggle.json ~/.kaggle/

In [ ]:
! kaggle datasets download -d gpiosenka/100-bird-species

100% 1.68G/1.69G [00:12<00:00, 193MB/s]
100% 1.69G/1.69G [00:12<00:00, 149MB/s]


In [ ]:
from zipfile import ZipFile
with ZipFile('/content/100-bird-species.zip','r') as zip:
  zip.extractall('/content/')

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions


In [ ]:
train_datagen = tf.keras.utils.image_dataset_from_directory(
    '/content/train',
    labels='inferred',
    label_mode='int',
    batch_size=32,
    image_size=(224,224)
)

test_datagen = tf.keras.utils.image_dataset_from_directory(
    '/content/test',
    labels='inferred',
    label_mode='int',
    batch_size=32,
    image_size=(224,224)
)

Found 70626 files belonging to 450 classes.
Found 2250 files belonging to 450 classes.


In [ ]:
def normalize(img,label):
  img = tf.cast(img/255., dtype=tf.float32)
  return img,label

train_datagen_normalized = train_datagen.map(normalize)
test_datagen_normalized = test_datagen.map(normalize)


In [ ]:
conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(224,224,3)
                  )

58889256/58889256 [==============================] - 1s 0us/step


In [ ]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten

In [ ]:
model=Sequential()
model.add(conv_base)
model.add(Flatten())
model.add(Dense(1024,activation='relu'))
model.add(Dense(512,activation='relu'))
model.add(Dense(450,activation='softmax'))

In [ ]:
conv_base.trainable=False

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 1024)              25691136  
                                                                 
 dense_1 (Dense)             (None, 512)               524800    
                                                                 
 dense_2 (Dense)             (None, 450)               230850    
                                                                 
Total params: 41,161,474
Trainable params: 26,446,786
Non-trainable params: 14,714,688
_________________________________________________________________


In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy']
              )

In [ ]:
history = model.fit(train_datagen_normalized, epochs=5,
                    validation_data=test_datagen_normalized)

Epoch 1/5
2208/2208 [==============================] - 398s 180ms/step - loss: 1.1185 - accuracy: 0.6894 - val_loss: 1.6905 - val_accuracy: 0.6044
Epoch 2/5
2208/2208 [==============================] - 398s 180ms/step - loss: 0.9283 - accuracy: 0.7347 - val_loss: 1.7551 - val_accuracy: 0.6084
Epoch 3/5
2208/2208 [==============================] - 398s 180ms/step - loss: 0.7793 - accuracy: 0.7712 - val_loss: 1.9045 - val_accuracy: 0.5707
Epoch 4/5
2208/2208 [==============================] - 398s 180ms/step - loss: 0.6730 - accuracy: 0.7973 - val_loss: 2.1423 - val_accuracy: 0.5702
Epoch 5/5
2208/2208 [==============================] - 398s 180ms/step - loss: 0.5832 - accuracy: 0.8206 - val_loss: 2.1133 - val_accuracy: 0.6053


In [ ]:
model.save('/content/drive/MyDrive/Colab Notebooks/saved_models/model1.h')